# Pipeline per estrarre tutte le tabelle e le immagini da un pdf e poi tutte le immagini e pdf estratti vengono processati per effettuare ocr tramite ollama modello granite. lo fa per tutti i file dentro una cartella

In [9]:
import logging
import os
import time
from pathlib import Path

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 2.0

def main():
    logging.basicConfig(level=logging.INFO)

    # cartella PDF in input
    pdf_folder = Path("/storage/data_4T_b/andreacutuli/PROVA/Documents/pdf_fac_simile")
    # cartella radice dove verranno create le sottocartelle di output
    output_root = Path("/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images")

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time_global = time.time()

    # Itera su tutti i PDF nella cartella
    for input_doc_path in pdf_folder.glob("*.pdf"):
        if not input_doc_path.exists():
            _log.warning(f"PDF non trovato: {input_doc_path}")
            continue

        # Crea la sottocartella di output per questo file
        doc_filename = input_doc_path.stem
        output_dir = output_root / doc_filename
        output_dir.mkdir(parents=True, exist_ok=True)

        start_time = time.time()

        conv_res = doc_converter.convert(str(input_doc_path))

        # --- Da qui in poi è TUTTO il tuo codice originale, invariato ---

        # Save page images
        pages_obj = getattr(conv_res.document, "pages", None)
        if pages_obj is None:
            _log.warning("Nessuna proprietà pages trovata su conv_res.document")
        else:
            try:
                iterator = list(pages_obj.items())  # dict-like
                is_dict = True
            except Exception:
                iterator = list(enumerate(pages_obj, start=1))  # list-like
                is_dict = False

            for key, page in iterator:
                page_no = getattr(page, "page_no", None) or (key if is_dict else key)
                page_image_filename = output_dir / f"{doc_filename}-page-{page_no}.png"
                try:
                    page_image = getattr(page, "image", None)
                    if page_image is None:
                        _log.debug(f"Nessuna immagine pagina per page {page_no}")
                        continue
                    pil_img = getattr(page_image, "pil_image", None)
                    if pil_img is not None:
                        pil_img.save(page_image_filename, format="PNG")
                    else:
                        page_image.save(str(page_image_filename), format="PNG")
                except Exception as e:
                    _log.exception(f"Errore salvataggio immagine pagina {page_no}: {e}")

        # Save images of figures and tables
        table_counter = 0
        picture_counter = 0
        try:
            iterator = conv_res.document.iterate_items()
        except Exception:
            iterator = []

        for element, _level in iterator:
            try:
                if isinstance(element, TableItem):
                    table_counter += 1
                    element_image_filename = output_dir / f"{doc_filename}-table-{table_counter}.png"
                    img = element.get_image(conv_res.document)
                    if hasattr(img, "save"):
                        img.save(element_image_filename, format="PNG")
                    else:
                        with open(element_image_filename, "wb") as fp:
                            fp.write(img)
                elif isinstance(element, PictureItem):
                    picture_counter += 1
                    element_image_filename = output_dir / f"{doc_filename}-picture-{picture_counter}.png"
                    img = element.get_image(conv_res.document)
                    if hasattr(img, "save"):
                        img.save(element_image_filename, format="PNG")
                    else:
                        with open(element_image_filename, "wb") as fp:
                            fp.write(img)
            except Exception as e:
                _log.exception(f"Errore salvataggio elemento {type(element)}: {e}")

        # Generazione manuale di markdown con segnaposto
        md_lines = []
        table_counter_md = 0
        picture_counter_md = 0

        try:
            iterator = conv_res.document.iterate_items()
        except Exception:
            iterator = []

        for element, _level in iterator:
            try:
                if isinstance(element, TableItem):
                    table_counter_md += 1
                    md_lines.append(f"[[TABLE-{table_counter_md}]]\n")
                elif isinstance(element, PictureItem):
                    picture_counter_md += 1
                    md_lines.append(f"[[IMAGE-{picture_counter_md}]]\n")
                else:
                    testo = getattr(element, "text", "") or getattr(element, "get_text", lambda: "")()
                    if testo:
                        md_lines.append(testo + "\n")
            except Exception as e:
                _log.exception(f"Errore processando elemento {type(element)}: {e}")

        md_placeholder_filename = output_dir / f"{doc_filename}-with-placeholders.md"
        try:
            with open(md_placeholder_filename, "w", encoding="utf-8") as f:
                f.write("\n".join(md_lines))
            print(f"Markdown con segnaposto salvato in: {md_placeholder_filename}")
        except Exception as e:
            _log.exception(f"Errore salvataggio markdown con segnaposto: {e}")

        end_time = time.time() - start_time
        _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")
        _log.info(f"Saved pages: {page_no if 'page_no' in locals() else 'n/a'}, tables: {table_counter}, pictures: {picture_counter}")

        for nome_file in os.listdir(output_dir):
            percorso_file = os.path.join(output_dir, nome_file)
            if os.path.isfile(percorso_file) and "page" in nome_file:
                os.remove(percorso_file)
                print(f"Cancellato: {nome_file}")

    total_time = time.time() - start_time_global
    _log.info(f"Tutti i PDF elaborati in {total_time:.2f} secondi.")


if __name__ == "__main__":
    main()


2025-10-01 12:26:08,085 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-01 12:26:08,092 - INFO - Going to convert document batch...
2025-10-01 12:26:08,093 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 85ac70aecbb3e9d18a585ddbe6342108
2025-10-01 12:26:08,094 - INFO - Accelerator device: 'cuda:0'
2025-10-01 12:26:12,326 - INFO - Accelerator device: 'cuda:0'
2025-10-01 12:26:14,662 - INFO - Accelerator device: 'cuda:0'
2025-10-01 12:26:16,589 - INFO - Processing document FAC-SIMILE 19.pdf
2025-10-01 12:26:31,297 - INFO - Finished converting document FAC-SIMILE 19.pdf in 23.21 sec.
2025-10-01 12:26:33,368 - INFO - Document converted and figures exported in 25.28 seconds.
2025-10-01 12:26:33,369 - INFO - Saved pages: 13, tables: 7, pictures: 3
2025-10-01 12:26:33,373 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-01 12:26:33,377 - INFO - Going to convert document batch...
2025-10-01 12:26:33,378 - INFO - Processing document FAC-SIMILE

Markdown con segnaposto salvato in: /storage/data_4T_b/andreacutuli/PROVA/Documents/output_images/FAC-SIMILE 19/FAC-SIMILE 19-with-placeholders.md
Cancellato: FAC-SIMILE 19-page-5.png
Cancellato: FAC-SIMILE 19-page-3.png
Cancellato: FAC-SIMILE 19-page-10.png
Cancellato: FAC-SIMILE 19-page-13.png
Cancellato: FAC-SIMILE 19-page-11.png
Cancellato: FAC-SIMILE 19-page-1.png
Cancellato: FAC-SIMILE 19-page-4.png
Cancellato: FAC-SIMILE 19-page-9.png
Cancellato: FAC-SIMILE 19-page-7.png
Cancellato: FAC-SIMILE 19-page-6.png
Cancellato: FAC-SIMILE 19-page-2.png
Cancellato: FAC-SIMILE 19-page-12.png
Cancellato: FAC-SIMILE 19-page-8.png


2025-10-01 12:26:51,122 - INFO - Finished converting document FAC-SIMILE 1.pdf in 17.75 sec.
2025-10-01 12:26:53,813 - INFO - Document converted and figures exported in 20.44 seconds.
2025-10-01 12:26:53,843 - INFO - Saved pages: 19, tables: 4, pictures: 1
2025-10-01 12:26:53,848 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-01 12:26:53,852 - INFO - Going to convert document batch...
2025-10-01 12:26:53,853 - INFO - Processing document FAC-SIMILE 3.pdf


Markdown con segnaposto salvato in: /storage/data_4T_b/andreacutuli/PROVA/Documents/output_images/FAC-SIMILE 1/FAC-SIMILE 1-with-placeholders.md
Cancellato: FAC-SIMILE 1-page-19.png
Cancellato: FAC-SIMILE 1-page-18.png
Cancellato: FAC-SIMILE 1-page-4.png
Cancellato: FAC-SIMILE 1-page-15.png
Cancellato: FAC-SIMILE 1-page-5.png
Cancellato: FAC-SIMILE 1-page-6.png
Cancellato: FAC-SIMILE 1-page-17.png
Cancellato: FAC-SIMILE 1-page-1.png
Cancellato: FAC-SIMILE 1-page-2.png
Cancellato: FAC-SIMILE 1-page-11.png
Cancellato: FAC-SIMILE 1-page-3.png
Cancellato: FAC-SIMILE 1-page-13.png
Cancellato: FAC-SIMILE 1-page-9.png
Cancellato: FAC-SIMILE 1-page-14.png
Cancellato: FAC-SIMILE 1-page-8.png
Cancellato: FAC-SIMILE 1-page-10.png
Cancellato: FAC-SIMILE 1-page-12.png
Cancellato: FAC-SIMILE 1-page-7.png
Cancellato: FAC-SIMILE 1-page-16.png


2025-10-01 12:27:09,939 - INFO - Finished converting document FAC-SIMILE 3.pdf in 16.09 sec.
2025-10-01 12:27:12,488 - INFO - Document converted and figures exported in 18.64 seconds.
2025-10-01 12:27:12,539 - INFO - Saved pages: 17, tables: 3, pictures: 1
2025-10-01 12:27:12,544 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-01 12:27:12,547 - INFO - Going to convert document batch...
2025-10-01 12:27:12,548 - INFO - Processing document FAC-SIMILE 21.pdf


...

Markdown con segnaposto salvato in: /storage/data_4T_b/andreacutuli/PROVA/Documents/output_images/FAC-SIMILE 6/FAC-SIMILE 6-with-placeholders.md
Cancellato: FAC-SIMILE 6-page-3.png
Cancellato: FAC-SIMILE 6-page-1.png
Cancellato: FAC-SIMILE 6-page-2.png


2025-10-01 12:29:35,116 - INFO - Finished converting document FAC-SIMILE 9.pdf in 5.84 sec.
2025-10-01 12:29:36,171 - INFO - Document converted and figures exported in 6.90 seconds.
2025-10-01 12:29:36,180 - INFO - Saved pages: 7, tables: 0, pictures: 0
2025-10-01 12:29:36,183 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-01 12:29:36,187 - INFO - Going to convert document batch...
2025-10-01 12:29:36,188 - INFO - Processing document FAC-SIMILE 13.pdf


Markdown con segnaposto salvato in: /storage/data_4T_b/andreacutuli/PROVA/Documents/output_images/FAC-SIMILE 9/FAC-SIMILE 9-with-placeholders.md
Cancellato: FAC-SIMILE 9-page-6.png
Cancellato: FAC-SIMILE 9-page-4.png
Cancellato: FAC-SIMILE 9-page-2.png
Cancellato: FAC-SIMILE 9-page-5.png
Cancellato: FAC-SIMILE 9-page-7.png
Cancellato: FAC-SIMILE 9-page-1.png
Cancellato: FAC-SIMILE 9-page-3.png


2025-10-01 12:29:46,736 - INFO - Finished converting document FAC-SIMILE 13.pdf in 10.55 sec.
2025-10-01 12:29:48,874 - INFO - Document converted and figures exported in 12.69 seconds.
2025-10-01 12:29:48,901 - INFO - Saved pages: 11, tables: 1, pictures: 1
2025-10-01 12:29:48,904 - INFO - Tutti i PDF elaborati in 220.82 secondi.


Markdown con segnaposto salvato in: /storage/data_4T_b/andreacutuli/PROVA/Documents/output_images/FAC-SIMILE 13/FAC-SIMILE 13-with-placeholders.md
Cancellato: FAC-SIMILE 13-page-9.png
Cancellato: FAC-SIMILE 13-page-11.png
Cancellato: FAC-SIMILE 13-page-8.png
Cancellato: FAC-SIMILE 13-page-10.png
Cancellato: FAC-SIMILE 13-page-7.png
Cancellato: FAC-SIMILE 13-page-6.png
Cancellato: FAC-SIMILE 13-page-5.png
Cancellato: FAC-SIMILE 13-page-2.png
Cancellato: FAC-SIMILE 13-page-1.png
Cancellato: FAC-SIMILE 13-page-3.png
Cancellato: FAC-SIMILE 13-page-4.png


In [ ]:
#SPOSTA TUTTI I FILE MARKDOWN IN UNA CARTELLA A PARTE
import os
import shutil

# Cartella sorgente
source_dir = "/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images"

# Cartella di destinazione
dest_dir = "/storage/data_4T_b/andreacutuli/PROVA/Documents/markdown_placeholders"
os.makedirs(dest_dir, exist_ok=True)

# Itera ricorsivamente tutte le sottocartelle
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(".md"):
            source_path = os.path.join(root, file)
            dest_path = os.path.join(dest_dir, file)

            # Sposta il file (sovrascrive se esiste già)
            shutil.move(source_path, dest_path)
            print(f"Spostato: {source_path} -> {dest_path}")


In [ ]:
import os
import ollama

input_root = "/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images"
descrizioni_root = "/storage/data_4T_b/andreacutuli/PROVA/Documents/descrizioni"
os.makedirs(descrizioni_root, exist_ok=True)

# Modello Ollama
model = "granite3.2-vision"
prompt = "Describe what's in this image."

# Itera sulle sottocartelle nella cartella principale di input
for subfolder in os.listdir(input_root):
    subfolder_path = os.path.join(input_root, subfolder)
    if not os.path.isdir(subfolder_path):
        continue  # salta eventuali file direttamente in input_root

    # Crea la sottocartella corrispondente per le descrizioni
    output_subfolder = os.path.join(descrizioni_root, subfolder)
    os.makedirs(output_subfolder, exist_ok=True)

    # Itera su ogni immagine nella sottocartella
    for nome_file in os.listdir(subfolder_path):
        percorso_file = os.path.join(subfolder_path, nome_file)

        if os.path.isfile(percorso_file) and nome_file.lower().endswith((".png", ".jpg", ".jpeg")):
            print(f"\nProcessing image: {nome_file} in folder {subfolder}")

            # Esegui inferenza con Ollama
            response = ollama.generate(
                model=model,
                prompt=prompt,
                images=[percorso_file]
            )

            descrizione = response['response']
            print("Description:", descrizione)

            # Salva la descrizione in file .txt
            txt_filename = os.path.splitext(nome_file)[0] + ".txt"
            txt_path = os.path.join(output_subfolder, txt_filename)
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(descrizione)
            print(f"[OK] Descrizione salvata in {txt_path}")



Processing image: FAC-SIMILE 5-table-1.png in folder FAC-SIMILE 5


2025-10-01 12:31:46,826 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image depicts a table that lists various financial activities along with their respective amounts in euros (€). Here is a detailed description of the table:

### Table Description

**Table Title:** Not explicitly provided, but it appears to be related to financial activities or expenses.

| Descrizione | Importo (€) |
|-----------------|---------------|
| Consolenza strategica finanziaria (Attività 1) | 5.000         |
| Supporto valutazioni progetti (Attività 2) | 6.500         |
| Consolenza finanza agevolata (Attività 3) | 4.500         |
| Totale           | 16.000        |

### Analysis and Description:

The table is structured with two columns: "Descrizione" and "Importo (€)." The "Descrizione" column lists the financial activities or expenses, while the "Importo (€)" column provides the corresponding amounts in euros. 

1. **Consolenza strategica finanziaria (Attività 1):**
   - This activity is described as a "strategic financial consolidation" and has an amou

2025-10-01 12:32:55,475 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image is a table that outlines various financial goals and their corresponding bonus amounts. Here is a detailed description of the table:

### Table Description

**Table Title:** Obiettivo raggiunto

**Columns:**
1. **Obiettivo raggiunto** (Goal Achieved)
2. **Bonus lordo (€)** (Bonus Amount in Euros)
3. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
4. **10 gol realizzati in Serie A** (10 goals scored in Serie A)
5. **20 gol realizzati in Serie A** (20 goals scored in Serie A)
6. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
7. **100.000 euro** (100,000 Euros)
8. **100.000 euro** (100,000 Euros)
9. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
10. **100.000 euro** (100,000 Euros)

### Analysis and Description:

The table is structured to outline different financial goals related to football (socc

...

Description: 
The image depicts a bar chart with six horizontal bars, each representing different categories or groups labeled from "WPs" to "Mese 6." The x-axis is labeled as "3," "4," "5," and "6," while the y-axis represents numerical values ranging from 1.1 to 3.1 in increments of 0.2.

Each bar corresponds to a specific category, with the height of the bar indicating the value associated with that category. The categories are as follows:

1. **WPs** (Women's Participation) - This category has two bars:
   - The first bar is at approximately 1.2.
   - The second bar is at approximately 1.3.

2. **Mese 1** - This category also has two bars:
   - The first bar is at approximately 1.4.
   - The second bar is at approximately 1.5.

3. **Mese 2** - This category has one bar:
   - The bar is at approximately 1.6.

4. **Mese 3** - This category has two bars:
   - The first bar is at approximately 1.7.
   - The second bar is at approximately 1.8.

5. **Mese 4** - This category has one bar:

2025-10-01 12:50:46,133 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image appears to be a scanned page from a document or report. It is divided into several sections with headings and subheadings that seem to outline various aspects of an intervention project related to energy efficiency, sustainability, and renewable energy systems. Here's a detailed breakdown of the content:

### Title Section
- **Title:** "Studio di Fattibilità e Progettazione dell'Intervento di Riconversione Energetica Sostenibile"
  - This translates to "Study on Feasibility and Design of the Renewable Energy Conversion Intervention."

### Table of Contents
- **1. Introduzione**
  - This section likely introduces the document, providing an overview or background information about the project.

- **2. Specifiche di Progetto**
  - This section seems to detail specific aspects of the project, such as objectives, activities, and methodology.

- **3. Organizzazione del Progetto**
  - This part likely discusses the organizational structure or framework for the project.

2025-10-01 12:51:11,230 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image displays a table with two columns labeled "Attività" (Activities) and "Costo [€]" (Cost [€]). The table is structured as follows:

| Attività | Costo [€] |
|----------|-------------|
| 1        | 35.000 €    |
| 2        | 45.000 €    |
| 3        | 40.000 €    |

### Detailed Description:

- **Title**: The table does not have a title, but it is labeled "Attività" and "Costo [€]". This suggests that the table is likely part of a report or analysis related to activities and their associated costs.

- **Columns**:
  - **Attività (Activities)**: This column lists different activities. The activities are numbered from 1 to 3, indicating a sequence or categorization of these activities.
  - **Costo [€] (Cost [€])**: This column lists the costs associated with each activity. Each cost is given in euros and represents the monetary value for that particular activity.

### Analysis:

1. **Activity 1**: The cost associated with Activity 1 is €35,000. This suggests a signi